In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import pytest
from pyomo.environ import (ConcreteModel,
                           Objective,
                           SolverStatus,
                           TerminationCondition,
                           units as pyunits)
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util import get_solver

from idaes.core import VaporPhase, Component
from idaes.generic_models.properties.core.state_definitions import FTPx
from idaes.generic_models.properties.core.eos.ideal import Ideal
import idaes.generic_models.properties.core.pure.RPP4 as RPP4

from idaes.generic_models.unit_models import Heater
from idaes.core import FlowsheetBlock

from idaes.generic_models.properties.core.generic.generic_property import \
        GenericParameterBlock

from idaes.core.util.utility_minimization import \
        min_utility, PinchDataClass, heat_ex_data, gen_curves

In [2]:
configuration = {
            # Specifying components
            "components": {

                'A': {"type": Component,
                      "enth_mol_ig_comp": RPP4,
                      "entr_mol_ig_comp": RPP4,
                      "parameter_data": {
                        "mw": (2.016E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (12.9e5, pyunits.Pa),  # [1]
                        "temperature_crit": (33.2, pyunits.K),  # [1]
                        "omega": -0.218,
                        "cp_mol_ig_comp_coeff": {
                            "A": (1000000, pyunits.J/pyunits.mol/pyunits.K),
                            "B": (0, pyunits.J/pyunits.mol/pyunits.K**2),
                            "C": (0, pyunits.J/pyunits.mol/pyunits.K**3),
                            "D": (0, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "entr_mol_form_vap_comp_ref": (
                            0, pyunits.J/pyunits.mol/pyunits.K),
                        "enth_mol_form_vap_comp_ref": (
                            0.0, pyunits.J/pyunits.mol)}}},

            # Specifying phases
            "phases":  {'Vap': {"type": VaporPhase,
                                "equation_of_state": Ideal}},

            # Set base units of measurement
            "base_units": {"time": pyunits.s,
                           "length": pyunits.m,
                           "mass": pyunits.kg,
                           "amount": pyunits.mol,
                           "temperature": pyunits.K},

            # Specifying state definition
            "state_definition": FTPx,
            "state_bounds": {"flow_mol": (0, 100, 1000, pyunits.mol/pyunits.s),
                             "temperature": (273.15, 300, 450, pyunits.K),
                             "pressure": (5e4, 1e5, 1e6, pyunits.Pa)},
            "pressure_ref": (1e5, pyunits.Pa),
            "temperature_ref": (300, pyunits.K)}

In [3]:
model = ConcreteModel()

model.fs = FlowsheetBlock(default={"dynamic": False})

# Add properties parameter blocks to the flowsheet with specifications
model.fs.props = GenericParameterBlock(default=configuration)

# Create an instance of the units, attaching them to the flowsheet
# Specify that the property package to be used with with eash unit.
model.fs.H101 = Heater(default={"property_package": model.fs.props,
                                "has_pressure_change": False,
                                "has_phase_equilibrium": False})

model.fs.H102 = Heater(default={"property_package": model.fs.props,
                                "has_pressure_change": False,
                                "has_phase_equilibrium": False})

model.fs.H103 = Heater(default={"property_package": model.fs.props,
                                "has_pressure_change": False,
                                "has_phase_equilibrium": False})

model.fs.H104 = Heater(default={"property_package": model.fs.props,
                                "has_pressure_change": False,
                                "has_phase_equilibrium": False})

model.fs.H101.inlet.mole_frac_comp[0, "A"].fix(1)
model.fs.H101.inlet.flow_mol[0].fix(1.5)
model.fs.H101.inlet.temperature.fix(160)
model.fs.H101.inlet.pressure.fix(100000)

model.fs.H101.outlet.temperature.fix(400)

model.fs.H102.inlet.mole_frac_comp[0, "A"].fix(1)
model.fs.H102.inlet.flow_mol[0].fix(1.3)
model.fs.H102.inlet.temperature.fix(100)
model.fs.H102.inlet.pressure.fix(100000)

model.fs.H102.outlet.temperature.fix(250)

model.fs.H103.inlet.mole_frac_comp[0, "A"].fix(1)
model.fs.H103.inlet.flow_mol[0].fix(1)
model.fs.H103.inlet.temperature.fix(400)
model.fs.H103.inlet.pressure.fix(100000)

model.fs.H103.outlet.temperature.fix(120)

model.fs.H104.inlet.mole_frac_comp[0, "A"].fix(1)
model.fs.H104.inlet.flow_mol[0].fix(2)
model.fs.H104.inlet.temperature.fix(340)
model.fs.H104.inlet.pressure.fix(100000)

model.fs.H104.outlet.temperature.fix(120)


In [4]:
min_utility(model.fs, [model.fs.H101, model.fs.H102],
            [model.fs.H103, model.fs.H104], 20)

In [5]:
model.fs.objective = Objective(expr=model.fs.Qs+model.fs.Qw)

In [6]:
solver = SolverFactory('ipopt')

In [7]:
results = solver.solve(model)

In [8]:
from idaes.core.util.utility_minimization import print_HX_results

In [9]:
print_HX_results(model.fs, [model.fs.H101, model.fs.H102,
            model.fs.H103, model.fs.H104])

Heat Exchanger Summary: 
Heat exchanger:  fs.H101
Inlet T:      160.000 K
Outlet T:     400.000 K
Q :            360000000.000 kg*m**2/s**3
Heat exchanger:  fs.H102
Inlet T:      100.000 K
Outlet T:     250.000 K
Q :            195000000.000 kg*m**2/s**3
Heat exchanger:  fs.H103
Inlet T:      400.000 K
Outlet T:     120.000 K
Q :           -280000000.000 kg*m**2/s**3
Heat exchanger:  fs.H104
Inlet T:      340.000 K
Outlet T:     120.000 K
Q :           -440000000.000 kg*m**2/s**3
